### Question 3: Count records

How many taxi trips were there on the 15th of October?



In [1]:
import pyspark
from pyspark.sql import SparkSession, types

In [2]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

25/03/07 15:20:05 WARN Utils: Your hostname, SOLIDCAD-SERVER resolves to a loopback address: 127.0.1.1; using 192.168.160.37 instead (on interface eth0)
25/03/07 15:20:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/07 15:20:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/07 15:20:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/07 15:20:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
df = spark.read \
     .option("header", "true") \
     .parquet('/home/myothet/repos/data-engineering/batch/yellow_tripdata_2024-10.parquet')

In [5]:
df.write.parquet('yellow_tripdata/2024/10', mode="overwrite")

In [13]:
df = spark.read.parquet('yellow_tripdata/2024/10')

In [15]:
df.count()

3833771

In [7]:
from pyspark.sql import functions as F

In [9]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [17]:
df \
    .withColumn('pickup_date', F.to_date(df.tpep_pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.tpep_dropoff_datetime)) \
    .filter("pickup_date = '2024-10-15'") \
    .filter("dropoff_date = '2024-10-15'") \
    .count()

127993

In [19]:
df.registerTempTable('yellow_tripdata_2024_10')

In [26]:
query = """
SELECT
    COUNT(1)
FROM 
    yellow_tripdata_2024_10
WHERE
    to_date(tpep_pickup_datetime) = '2024-10-15' AND
    to_date(tpep_dropoff_datetime) = '2024-10-15';
"""

In [27]:
spark.sql(query).show()

+--------+
|count(1)|
+--------+
|  127993|
+--------+

